In [1]:
import os
import tensorflow as tf
os.environ['KERAS_BACKEND'] = 'tensorflow'

import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img
        
PATH = 'input'

train_df = pd.read_csv('input/train.csv')
train_ids = train_df.id.values

sample_df = pd.read_csv('input/sample_submission.csv')
test_ids = sample_df.id.values

Using TensorFlow backend.


In [2]:
im_width = 128
im_height = 128
im_chan = 1

def load_and_resize(path):
    img = load_img(path)  
    x = img_to_array(img)[:,:,1]
    if x.shape != (101, 101):
        print(path, shape)
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    return x

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.float32)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):    
    X_train[n] = load_and_resize(os.path.join(PATH, 'train', 'images', "%s.png" % id_))
    Y_train[n] = load_and_resize(os.path.join(PATH, 'train', 'masks', "%s.png" % id_))

X_train /= 255.0 
X_train -= .5
print('Done!')

  0%|          | 0/4000 [00:00<?, ?it/s]

Getting and resizing train images and masks ... 


/opt/miniconda2/envs/avito/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|██████████| 4000/4000 [00:19<00:00, 209.13it/s]


Done!


In [5]:
# Get and resize test images
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.float32)

print('Getting and resizing test images ... ')

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    X_test[n] = load_and_resize(os.path.join(PATH, 'test', 'images', '%s.png' % id_))
    
X_test /= 255.0    
X_test -= .5
print('Done!')

  0%|          | 0/18000 [00:00<?, ?it/s]/opt/miniconda2/envs/avito/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
  0%|          | 37/18000 [00:00<00:48, 367.25it/s]

Getting and resizing test images ... 


100%|██████████| 18000/18000 [00:27<00:00, 653.70it/s]


Done!


In [6]:
X_train.shape, Y_train.shape, X_test.shape

((4000, 128, 128, 1), (4000, 128, 128, 1), (18000, 128, 128, 1))

In [9]:
np.save('input/X_train.npy', X_train)
np.save('input/y_train.npy', Y_train)
np.save('input/X_test.npy', X_test)

In [3]:
def load_extra():
    depths_df = pd.read_csv('input/depths.csv')
    z_dict = depths_df.set_index('id').to_dict()['z']
    X_train = np.array([z_dict[x] for x in train_ids])
    X_test = np.array([z_dict[x] for x in test_ids])
    
    z_mean = np.mean(X_train)
    z_std = np.std(X_train)
    
    X_train -= z_mean
    X_train /= z_std
    
    X_test -= z_mean
    X_test /= z_std
    
    return X_train, X_test

X_train, X_test = load_extra()
